In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
edges_df = pd.read_csv('gephi_edges.csv')
edges_df = edges_df.dropna()
edges_df['target'] = edges_df['target'].astype('int')

In [5]:
G = nx.from_pandas_edgelist(edges_df, source='source', target='target', edge_attr='semester')

In [126]:
title = 'Weird Fiction Discussion Board Visualization'

HOVER_TOOLTIPS = [('Poster', '@index')]

plot = figure(tooltips=HOVER_TOOLTIPS,
    tools='pan, wheel_zoom, save, reset', active_scroll='wheel_zoom',
    x_range = Range1d(-50, 50), y_range = Range1d(-50, 50), title=title)

network_graph = from_networkx(G, nx.spring_layout, scale=50, center=(0,0))

network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)
show(plot)

In [10]:
title = 'Weird Fiction Discussion Board Visualization'

HOVER_TOOLTIPS = [('Poster', '@index')]

plot = figure(tooltips=HOVER_TOOLTIPS,
    tools='pan, wheel_zoom, save, reset', active_scroll='wheel_zoom',
    x_range = Range1d(-50, 50), y_range = Range1d(-50, 50), title=title)

network_graph = from_networkx(G, nx.spiral_layout, scale=50, center=(0,0))

network_graph.node_renderer.glyph = Circle(size=15, fill_color='mediumorchid')

network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)
show(plot)

In [127]:
degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees)
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [128]:
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

In [129]:
color_palette = Purples8

In [130]:
HOVER_TOOLTIPS_1 = [('Poster', '@index'), ('Degree', '@degree')]
plot_1 = figure(tooltips=HOVER_TOOLTIPS_1,
    tools='pan, wheel_zoom, save, reset', active_scroll='wheel_zoom',
    x_range = Range1d(-50, 50), y_range = Range1d(-50, 50), title=title)

network_graph = from_networkx(G, nx.spring_layout, scale=50, center=(0,0))
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot_1.renderers.append(network_graph)

show(plot_1)


In [131]:
from networkx.algorithms import community

In [132]:
communities = community.greedy_modularity_communities(G)

In [133]:
modularity_class = {}
modularity_color = dict()

In [134]:
for community_number, community in enumerate(communities):
    for name in community:
        modularity_class[name] = community_number
        modularity_color[name] = Purples8[community_number]

In [135]:
nx.set_node_attributes(G, modularity_class, 'modularity_class')
nx.set_node_attributes(G, modularity_color, 'modularity_color')

In [136]:
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'
#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Purples8
#Choose a title!
title = 'Weird Fiction Discussion Board Visualization'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Poster", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset, tap", active_scroll='wheel_zoom',
            x_range=Range1d(-50, 50), y_range=Range1d(-50, 50), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)

In [137]:
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Game of Thrones Network'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Poster", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)
#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

    #Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
#save(plot, filename=f"{title}.html")

In [145]:
SAME_SEMESTER, DIFFERENT_SEMESTER = 'navy', 'firebrick'

for i in sorted(G.nodes()):
    G.nodes[i]['semester'] = edges_df.semester[i]

edge_attr = {}
for start_node, end_node, _ in G.edges(data=True):
    edge_color = SAME_SEMESTER if G.nodes[start_node]['semester'] == G.nodes[end_node]['semester'] else DIFFERENT_SEMESTER
    edge_attr[(start_node, end_node)] = edge_color

nx.set_edge_attributes(G, edge_attr, 'edge_color')

HOVER_TOOLTIPS = [
       ("index", "@index"),
        ("semester", "@semester")
]
title = "With Semester Coloring"
#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)


g = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))
g.node_renderer.glyph = Circle(size=15, fill_color=Purples8[3])
g.edge_renderer.glyph = MultiLine(line_color='edge_color', line_alpha=0.7, line_width=1)

p.renderers.append(g)
show(p)